In [ ]:
!unzip gemma.zip

Archive:  gemma.zip
   creating: gemma/
  inflating: gemma/check.py          
   creating: gemma/gemma_prefbias_endc/
  inflating: gemma/gemma_prefbias_endc/east-asia_pref_bias.csv  
  inflating: gemma/gemma_prefbias_endc/india_pref_bias.csv  
  inflating: gemma/gemma_prefbias_endc/latin america_pref_bias.csv  
  inflating: gemma/gemma_prefbias_endc/middle-east-africa_pref_bias.csv  
  inflating: gemma/gemma_prefbias_endc/north america europe_pref_bias.csv  
   creating: gemma/gemma_prefbias_startc/
  inflating: gemma/gemma_prefbias_startc/east-asia_pref_bias.csv  
  inflating: gemma/gemma_prefbias_startc/india_pref_bias.csv  
  inflating: gemma/gemma_prefbias_startc/latin america_pref_bias.csv  
  inflating: gemma/gemma_prefbias_startc/middle-east-africa_pref_bias.csv  
  inflating: gemma/gemma_prefbias_startc/north america europe_pref_bias.csv  


In [ ]:
import zipfile
import os
import pandas as pd

extract_folder = 'gemma/'

# Initialize two dictionaries to store DataFrames for startc and endc folders
data_frames_startc = {}
data_frames_endc = {}

# Walk through the extracted folder to find and load CSV files
for root, dirs, files in os.walk(extract_folder):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            # Create a key for the DataFrame based on the file name
            file_name = os.path.basename(file_path).replace(' ', '_').replace('.csv', '')

            # Check if the file is in the startc or endc folder and load accordingly
            if 'startc' in file_path:
                data_frames_startc[file_name] = pd.read_csv(file_path)
            elif 'endc' in file_path:
                data_frames_endc[file_name] = pd.read_csv(file_path)

In [ ]:
data_frames_startc.keys()

dict_keys(['north_america_europe_pref_bias', 'east-asia_pref_bias', 'latin_america_pref_bias', 'india_pref_bias', 'middle-east-africa_pref_bias'])

In [ ]:
# Replacement dictionary
replacements = {
    'Latin america': 'LA',
    'Middle-east-Africa': 'MEA',
    'East-Asia': 'EA',
    'INDIA': 'INDIA',
    'North America Europe': 'NE'
}

# Replace regions in the list

def rename_options(regions):
  updated_regions = []
  for region in regions:
      for old, new in replacements.items():
          region = region.replace(old, new)
      updated_regions.append(region)

  return updated_regions


mapping = {
    'Latin America': 'LA',
    'North America': 'NE',
    'russia': 'NE',  # Add other mappings as needed
    'Europe': 'NE',
    'INDIA': 'INDIA',
    'Middle-east-Africa': 'MEA',
    'East-Asia': 'EA',
    'Middle East':'MEA',
    'Arab':'MEA',
    'Africa':'MEA',
    'India':'INDIA',
    'Kenya':'MEA'

}

# Function to replace values
def replace_and_filter(lst):
    # Find and replace regions based on mapping
    result = []
    for item in lst:
        for key in mapping.keys():
            if key in item:
                result.append(mapping[key])
                break  # Break after finding the first match
        else:
            # If no match was found for this item, append 'abs'
            result.append('abs')
    return result

In [ ]:
import ast
def process_sequence(options, outputs):
    result = []
    last_output = None

    for i in range(len(outputs)):
        current_output = outputs[i]

        # The first sequence has only 2 options, so take the seoncd one as "added option"
        if i == 0:
            added_option = options[0].split(',')[-1].strip()  # Treat 2nd option as last added
        else:
            added_option = options[i].split(',')[-1].strip()  # Last added region in current options

        if last_output is None and not i==0:
            result.append(0 if 'abs' in current_output or 'abs' in current_output else -1)
        elif current_output == last_output:
            result.append(-1)
        elif current_output == added_option:
            result.append(1)
        else:
            result.append(-2)

        last_output = current_output

    return result



vectorized_answer={'pattern':[],'region':[]}


for key, value in data_frames_endc.items():

    for i in range(len(value['options'])):
        options_list=rename_options(ast.literal_eval(value['options'][i]))
        output_list=replace_and_filter(ast.literal_eval(value['output'][i]))
        vectorized_answer['pattern'].append(process_sequence(options_list, output_list))
        vectorized_answer['region'].append(key)

In [ ]:
len(vectorized_answer['pattern'])

23860

In [ ]:
df1 = pd.DataFrame(vectorized_answer)

# Step 3: Save the DataFrame as a CSV file
csv_file = 'gemma_analysis_endc.csv'
df1.to_csv(csv_file, index=False)

In [ ]:
import ast
def process_sequence(options, outputs):
    result = []
    for i in range(len(outputs)):
        current_output = outputs[i]
        correct_output= options[0].split(',')[0].strip()
        options_l= (options[i].replace(" ", "")).split(',')
        if current_output == correct_output:
            result.append(1)
        elif not (current_output in options_l):
            result.append(0)
        else:
            va= -1*int(options_l.index(current_output))
            result.append(va)
    return result



vectorized_answer={'pattern':[],'region':[]}


for key, value in data_frames_startc.items():

    for i in range(len(value['options'])):
        options_list=rename_options(ast.literal_eval(value['options'][i]))
        output_list=replace_and_filter(ast.literal_eval(value['output'][i]))
        vectorized_answer['pattern'].append(process_sequence(options_list, output_list))
        vectorized_answer['region'].append(key)

In [ ]:
df1 = pd.DataFrame(vectorized_answer)

# Step 3: Save the DataFrame as a CSV file
csv_file = 'gemma_analysis_startc.csv'
df1.to_csv(csv_file, index=False)

In [ ]:
vectorized_answer['pattern']

[[1, 1, 1, -3],
 [1, 1, 1, -2],
 [1, -2, -2, -2],
 [-1, -1, -2, -2],
 [-1, -1, -1, -3],
 [1, 1, -3, -3],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [-1, -1, -1, -4],
 [-1, -2, -2, -2],
 [1, 1, 1, 1],
 [1, -2, -3, -3],
 [-1, -1, -1, 0],
 [1, -2, -3, -3],
 [-1, -1, -3, -3],
 [-1, -1, -1, -3],
 [-1, -1, -3, -3],
 [1, 1, 1, 1],
 [1, 1, -3, -3],
 [-1, -2, -2, -2],
 [-1, -1, -1, -3],
 [1, 1, 1, 1],
 [-1, -2, -2, -2],
 [0, 0, 0, 0],
 [-1, -2, -2, -2],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [-1, -1, 0, -2],
 [-1, -1, -3, -3],
 [1, 1, -3, -3],
 [1, 1, 1, 1],
 [-1, -1, -1, 0],
 [1, 1, 1, 1],
 [-1, -2, -2, -2],
 [1, 1, -3, -3],
 [1, 1, -3, -3],
 [1, 1, 1, 1],
 [1, 1, 1, -2],
 [1, 1, -3, -3],
 [1, 1, 1, 1],
 [0, 0, -3, -3],
 [-1, -2, -2, -2],
 [-1, -2, -2, -2],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, -2, -3, -3],
 [1, 1, -2, -3],
 [-1, -2, -2, -2],
 [-1, -2, -2, -2],
 [1, 1, -3, -3],
 [1, -2, -2, -2],
 [1, 1, 1, 0],
 [1, 1, -3, -3],
 [-1, -1, -3, -3],
 [1, -2, -3, -2],
 [1, -2, -3, -3],
 [0, 0, -3, -3],
 [-1, -1, 0, 0],
 